In [1]:
from langchain.globals import set_llm_cache
# from langchain_openai import OpenAI
from langchain_community.chat_models import ChatOllama

# To make the caching really obvious, lets use a slower model.
# llm = OpenAI(model_name="gpt-3.5-turbo-instruct", n=2, best_of=2)
model_local = ChatOllama(model="mistral",temperature=0)

In [2]:
from langchain.cache import InMemoryCache


In [3]:
set_llm_cache(InMemoryCache())

In [4]:
model_local.invoke("Tell me a joke")


AIMessage(content=" Why don't scientists trust atoms?\n\nBecause they make up everything!", response_metadata={'model': 'mistral', 'created_at': '2024-04-08T07:09:33.2889927Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 8511613700, 'load_duration': 6346988100, 'prompt_eval_count': 13, 'prompt_eval_duration': 711905000, 'eval_count': 17, 'eval_duration': 1440983000}, id='run-4b98566a-51d7-432f-9b7a-e05449f63c98-0')

In [5]:
model_local.invoke("Tell me a joke")

AIMessage(content=" Why don't scientists trust atoms?\n\nBecause they make up everything!", response_metadata={'model': 'mistral', 'created_at': '2024-04-08T07:09:33.2889927Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 8511613700, 'load_duration': 6346988100, 'prompt_eval_count': 13, 'prompt_eval_duration': 711905000, 'eval_count': 17, 'eval_duration': 1440983000}, id='run-4b98566a-51d7-432f-9b7a-e05449f63c98-0')

In [7]:
import streamlit as st
import pandas as pd
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import BSHTMLLoader
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_community.vectorstores import FAISS
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache

In [6]:
model_local = ChatOllama(model="mistral",temperature=0)
set_llm_cache(InMemoryCache())

In [8]:
embedding = embeddings.OllamaEmbeddings(model='nomic-embed-text')

vectorstore = FAISS.load_local("db_demos/test_9", embedding,allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever()

In [9]:
class format_json(BaseModel):
                university_name : str = Field(description="organization name from the given context")
                address: str = Field(description="address from the given context")
                email_address: str = Field(description="email address from the given context")
                contact_number: str = Field(description="contact number from the given context")

In [10]:
after_rag_template = """Answer the question based only on the following context:
            {context} and only provide these details in this order only and the response will always will be containing these informations only such as address,contact number,email address in a json format
            {format_instructions}
            Question: {question}
            """

In [14]:
parser = JsonOutputParser(pydantic_object=format_json)
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template,partial_variables={"format_instructions": parser.get_format_instructions()},)
after_rag_chain = (
                        {"context": retriever, "question": RunnablePassthrough()}
                        | after_rag_prompt
                        | model_local
                        | parser
                    )

In [15]:
after_rag_chain.invoke("provide the metioned details from the context and make sure you providing them from the given context")

{'university_name': 'Leiden University Medical Center (LUMC)',
 'address': 'Weg-van-Schenckstraat 25, 2311 GP Leiden, The Netherlands',
 'email_address': 'pers@lumc.nl',
 'contact_number': '+31 (0)6 11 37 11 46'}

In [20]:
after_rag_chain.invoke("provide the metioned details from the context and make sure you providing them from the given context")

{'university_name': 'Leiden University Medical Center (LUMC)',
 'address': 'Weg-van-Schenckstraat 25, 2311 GP Leiden, The Netherlands',
 'email_address': 'pers@lumc.nl',
 'contact_number': '+31 (0)6 11 37 11 46'}